In [ ]:
#!pip install bash_kernel
#!python -m bash_kernel.install

In [ ]:
#!sudo apt-get install man-db

In [ ]:
#!export MANPATH=/usr/share/man/man1:$MANPATH

# Create the internsctl Script

In [ ]:
%%bash

cat <<'EOF' > internsctl.sh

function display_manual() {
    echo "MANUAL PAGE FOR INTERNSCTL COMMAND"
    echo "---------------------------------"
    echo "The 'internsctl' command is a custom Linux command that provides various functionalities."
    echo ""
}

function display_help() {
    echo "USAGE:"
    echo "  internsctl [OPTIONS]"
    echo ""

    echo "OPTIONS:"
    echo "  --help                 Display help and usage examples"
    echo "  --version              Display command version"
    echo "  cpu getinfo            Get CPU information"
    echo "  memory getinfo         Get memory information"
    echo "  user create <username> Create a new user with the given username"
    echo "  user list              List all regular users"
    echo "  user list --sudo-only  List all users with sudo permissions"
    echo "  file getinfo <file-name>              Get information about a file"
    echo "  file getinfo [options] <file-name>    Get specific information about a file"
    echo ""
    echo "EXAMPLES:"
    echo "  internsctl --help                # Display help and usage examples"
    echo "  internsctl --version             # Display command version"
    echo "  internsctl cpu getinfo           # Get CPU information"
    echo "  internsctl memory getinfo        # Get memory information"
    echo "  internsctl user create john      # Create a new user with username 'john'"
    echo "  internsctl user list             # List all regular users"
    echo "  internsctl user list --sudo-only # List all users with sudo permissions"
    echo "  internsctl file getinfo myfile   # Get information about the file 'myfile'"
    echo "  internsctl file getinfo --size myfile    # Get the size of the file 'myfile'"
    echo "  internsctl file getinfo --permissions myfile   # Get file permissions for 'myfile'"
    echo "  internsctl file getinfo --owner myfile    # Get the owner of the file 'myfile'"
    echo "  internsctl file getinfo --last-modified myfile    # Get the last modified time of 'myfile'"
}

function display_version() {
    echo "internsctl version v0.1.0"
}

case "$1" in
    --help)
        display_help
        ;;
    --version)
        display_version
        ;;
    cpu)
        echo "CPU Information:"
        lscpu
        ;;
    memory)
        echo "Memory Information:"
        free
        ;;
    user)
         if [ "$2" == "create" ]; then
            if [ -z "$3" ]; then
                echo "Error: Please provide a username to create."
            else
                useradd "$3"
            fi
        elif [ "$2" == "list" ]; then
            if [ "$3" == "--sudo-only" ]; then
                echo "Users with sudo permissions:"
                grep -E 'sudo|admin' /etc/group | cut -d: -f4
            else
                echo "Regular users:"
                cut -d: -f1 /etc/passwd
            fi
        else
            echo "Invalid usage. See 'internsctl --help' for usage instructions."
        fi
        ;;
    file)
        if [ "$2" == "getinfo" ]; then
            if [ "$3" ]; then
                filename="$3"
                if [ -f "$filename" ]; then
                    if [ $# -eq 3 ]; then
                        echo "File Information for $filename:"
                        echo "File: $filename"
                        echo "Access: $(stat -c %A "$filename")"
                        echo "Size(B): $(stat -c %s "$filename")"
                        echo "Owner: $(stat -c %U "$filename")"
                        echo "Modify: $(stat -c %y "$filename")"
                    else
                        shift 3
                        while [ $# -gt 0 ]; do
                            case "$1" in
                                --size|-s)
                                    echo "Size(B): $(stat -c %s "$filename")"
                                    ;;
                                --permissions|-p)
                                    echo "Access: $(stat -c %A "$filename")"
                                    ;;
                                --owner|-o)
                                    echo "Owner: $(stat -c %U "$filename")"
                                    ;;
                                --last-modified|-m)
                                    echo "Modify: $(stat -c %y "$filename")"
                                    ;;
                                *)
                                    echo "Invalid option: $1"
                                    ;;
                            esac
                            shift
                        done
                    fi
                else
                    echo "Error: File '$filename' not found."
                fi
            else
                echo "Error: Please provide a file name."
            fi
        else
            echo "Invalid usage. See 'internsctl --help' for usage instructions."
        fi
        ;;
    *)
        echo "Invalid command: $1"
        ;;
esac

EOF


# Make the script executable
chmod +x internsctl.sh

mv internsctl.sh /usr/local/bin/internsctl

# Create the Manual Page File:

In [ ]:
'''
%%bash

#Create the Manual Page File internsctl.1
cat <<EOF > internsctl.1
.TH INTERNSCTL 1 "August 2023" "Version 0.1.0"
.SH NAME
internsctl \- A custom Linux command for various functionalities
.SH SYNOPSIS
.B internsctl
.RB [ \-\-help | \-\-version | cpu | memory | user ]
.RB [ cpu \fIgetinfo\fP | memory \fIgetinfo\fP ]
.RB [ user \fIcreate\fP \fI<username>\fP | user \fIlist\fP [ \-\-sudo\-only ] ]
.RB [ file \fIgetinfo\fP \fI<file\-name>\fP | file \fIgetinfo\fP [ \-\-size | \-\-permissions | \-\-owner | \-\-last\-modified ] \fI<file\-name>\fP ]
.SH DESCRIPTION
The \fBinternsctl\fP command provides various functionalities for system information and user management.
.SH OPTIONS
.TP
\fB\-\-help\fP
Display help and usage examples.
.TP
\fB\-\-version\fP
Display command version.
.SH COMMANDS
.TP
\fBcpu\fP
Get CPU information.
.TP
\fBmemory\fP
Get memory information.
.TP
\fBuser create <username>\fP
Create a new user with the given username.
.TP
\fBuser list\fP
List all regular users.
.TP
\fBuser list \-\-sudo\-only\fP
List all users with sudo permissions.
.TP
\fBfile getinfo <file\-name>\fP
Get information about a file.
.TP
\fBfile getinfo \-\-size <file\-name>\fP
Get the size of a file.
.TP
\fBfile getinfo \-\-permissions <file\-name>\fP
Get file permissions for a file.
.TP
\fBfile getinfo \-\-owner <file\-name>\fP
Get the owner of a file.
.TP
\fBfile getinfo \-\-last\-modified <file\-name>\fP
Get the last modified time of a file.
.SH EXAMPLES
Show usage examples of the \fBinternsctl\fP command.
EOF

#Install the Manual Page
sudo install -g 0 -o 0 -m 0644 internsctl.1 /usr/share/man/man1/

#Update the Manual Database so that the man command can find your manual page
sudo mandb
'''

'\n%%bash \n\n#Create the Manual Page File internsctl.1\ncat <<EOF > internsctl.1\n.TH INTERNSCTL 1 "August 2023" "Version 0.1.0"\n.SH NAME\ninternsctl \\- A custom Linux command for various functionalities\n.SH SYNOPSIS\n.B internsctl\n.RB [ \\-\\-help | \\-\\-version | cpu | memory | user ]\n.RB [ cpu \x0cIgetinfo\x0cP | memory \x0cIgetinfo\x0cP ]\n.RB [ user \x0cIcreate\x0cP \x0cI<username>\x0cP | user \x0cIlist\x0cP [ \\-\\-sudo\\-only ] ]\n.RB [ file \x0cIgetinfo\x0cP \x0cI<file\\-name>\x0cP | file \x0cIgetinfo\x0cP [ \\-\\-size | \\-\\-permissions | \\-\\-owner | \\-\\-last\\-modified ] \x0cI<file\\-name>\x0cP ]\n.SH DESCRIPTION\nThe \x0cBinternsctl\x0cP command provides various functionalities for system information and user management.\n.SH OPTIONS\n.TP\n\x0cB\\-\\-help\x0cP\nDisplay help and usage examples.\n.TP\n\x0cB\\-\\-version\x0cP\nDisplay command version.\n.SH COMMANDS\n.TP\n\x0cBcpu\x0cP\nGet CPU information.\n.TP\n\x0cBmemory\x0cP\nGet memory information.\n.TP\n\x0cBu

In [ ]:
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


# Section A
## 1. man internsctl
## 2. internsctl --help
## 3. internsctl --version


In [ ]:
#!sudo unminimize

In [ ]:
#!man internsctl

In [ ]:
!internsctl --help

USAGE:
  internsctl [OPTIONS]

OPTIONS:
  --help                 Display help and usage examples
  --version              Display command version
  cpu getinfo            Get CPU information
  memory getinfo         Get memory information
  user create <username> Create a new user with the given username
  user list              List all regular users
  user list --sudo-only  List all users with sudo permissions
  file getinfo <file-name>              Get information about a file
  file getinfo [options] <file-name>    Get specific information about a file

EXAMPLES:
  internsctl --help                # Display help and usage examples
  internsctl --version             # Display command version
  internsctl cpu getinfo           # Get CPU information
  internsctl memory getinfo        # Get memory information
  internsctl user create john      # Create a new user with username 'john'
  internsctl user list             # List all regular users
  internsctl user list --sudo-only # List a

In [ ]:
# Test the help command
#!internsctl --help

# Test the version command
!internsctl --version

internsctl version v0.1.0


# Section B (Part1)
## 1. get cpu information
## 2. get memory information



In [ ]:
!internsctl cpu getinfo

CPU Information:
Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4400.38
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscal
                         l nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopo
                         logy nonstop_tsc cpuid tsc_known_freq pni pclmulqdq sss
                         e3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes 
                         xsave avx f16c rdrand hypervisor lahf

In [ ]:
!internsctl memory getinfo

Memory Information:
               total        used        free      shared  buff/cache   available
Mem:        13294252      659064     8993872        5284     3641316    12330852
Swap:              0           0           0


# Section B (Part2)
## create a new user on server

*    create new user

*   list regular users present on server

*   list all users with sudo permissions on server



In [ ]:
#%%bash
#sudo useradd aneeta

In [ ]:
#!internsctl user create aneeta
#!internsctl user create sunita
#!internsctl user create neelam

In [ ]:
!internsctl user list

Regular users:
root
daemon
bin
sys
sync
games
man
lp
mail
news
uucp
proxy
www-data
backup
list
irc
gnats
nobody
_apt
systemd-network
systemd-resolve
messagebus
aneeta
sunita
neelam


In [ ]:
!internsctl user list --sudo-only
# to grant permission in linux environment
#sudo usermod -aG sudo username

Users with sudo permissions:



# Section B (Part3)
## 1. get file information
## 2. specific information



In [ ]:
#!touch hello.txt
#!echo "Hello my name is Aneeta" > hello.txt or
#!echo "Hello my name is Aneeta" > hello.txt
!internsctl file getinfo hello.txt

File Information for hello.txt:
File: hello.txt
Access: -rw-r--r--
Size(B): 24
Owner: root
Modify: 2023-08-03 01:52:56.431642126 +0000


In [ ]:
!internsctl file getinfo hello.txt -s
#!internsctl file getinfo hello.txt --size


Size(B): 24


In [ ]:
!internsctl file getinfo hello.txt --permissions

Access: -rw-r--r--


In [ ]:
!internsctl file getinfo hello.txt --owner

Owner: root


In [ ]:
!internsctl file getinfo hello.txt --last-modified

Modify: 2023-08-03 01:52:56.431642126 +0000
